In [1]:
import numpy as np
import scipy.linalg
import strawberryfields as sf

In [2]:
def symplectic_typical(N, style="xxpp"):
    """
    生成一个典型的辛矩阵
    :param N: 模式数
    :param style: "xxpp" 或 "xpxp"
    :return: 2N x 2N 的辛矩阵
    """
    if style == "xxpp":
        I = np.eye(N)
        O = np.zeros((N, N))
        J = np.block([[O, I], [-I, O]])
        return J
    elif style == "xpxp":
        j2 = np.array([[0, 1], [-1, 0]])
        J = np.kron(np.eye(N), j2)
        return J
    else:
        raise ValueError("[symplectic_typical] style must be either 'xxpp' or 'xpxp'")

def permutation_matrix(N):
    """
    生成将变量排列从 xpxp → xxpp 的置换矩阵 P
    可用于变换变量排列方式：
        M_xxpp = P @ M_xpxp @ P.T
        M_xpxp = P.T @ M_xxpp @ P
    :param N: 模式数
    :return: 置换矩阵 P (2N x 2N)
    """
    P = np.zeros((2*N, 2*N))
    for i in range(N):
        P[i, 2*i] = 1
        P[N + i, 2*i + 1] = 1
    return P

def is_symplectic(S, style="xxpp", tol=1e-8, verbose=False):
    """
    检查矩阵 S 是否是辛矩阵，即 S.T @ J @ S ≈ J
    """
    if not isinstance(S, np.ndarray) or S.ndim != 2 or S.shape[0] != S.shape[1] or S.shape[0] % 2 != 0:
        raise ValueError("[is_symplectic] S 必须是偶数阶方阵")
    N = S.shape[0] // 2
    J = symplectic_typical(N, style=style)
    left = S.T @ J @ S
    is_symp = np.allclose(left, J, atol=tol)
    if verbose:
        print("S^T J S - J =\n", left - J)
    return is_symp

def is_physical_covariance(V, style="xxpp", hbar=1.0, tol=1e-10, verbose=False):
    N = V.shape[0] // 2
    J = symplectic_typical(N, style=style)
    matrix = V + 1j * hbar / 2 * J
    eigvals = np.linalg.eigvals(matrix)
    
    if verbose:
        print("最低特征值实部:", np.min(np.real(eigvals)))

    return np.all(np.real(eigvals) >= -tol)

def symplectic_eigenvalues(V, style="xxpp", hbar=1.0):
    N = V.shape[0] // 2
    J = symplectic_typical(N, style=style)
    eigvals = np.linalg.eigvals(1j * J @ V)
    return np.sort(np.abs(eigvals))[::2]

def is_physical_by_symplectic(V, style="xxpp", hbar=1.0, tol=1e-10, verbose=False):
    eigv = symplectic_eigenvalues(V, style=style, hbar=hbar)
    if verbose:
        print("辛特征值:", eigv)
        print("最低特征值:", np.min(eigv))
    return np.all(eigv >= hbar / 2 - tol)

# def decomp_williamson1(V, verbose=False):
    """
    威廉姆森分解：V = S D S^T
    返回 D（对角矩阵）和 S（辛矩阵）
    """
    if not isinstance(V, np.ndarray) or V.ndim != 2 or V.shape[0] != V.shape[1] or V.shape[0] % 2 != 0:
        raise ValueError("[decomp_williamson] 输入必须是偶数阶方阵")

    N = V.shape[0] // 2
    J = symplectic_typical(N, style="xxpp")

    # 计算辛特征值（对角项 ν_i）
    iJV = 1j * J @ V
    eigvals = np.linalg.eigvals(iJV)
    symplectic_eigs = np.sort(np.abs(eigvals))[::2]  # 保留非简并值

    # 构造 D = diag(ν_1, ..., ν_N, ν_1, ..., ν_N)
    D = np.diag(np.repeat(symplectic_eigs, 2))

    # 构造 S 使用 scipy 的 canonical form 方法
    S = scipy.linalg.sqrtm(V) @ scipy.linalg.inv(scipy.linalg.sqrtm(D))

    if verbose:
        print("辛特征值 ν_i:", symplectic_eigs)
        print("D:\n", D)
        print("S^T J S - J =\n", S.T @ J @ S - J)

    return D, S

# def decomp_williamson2(V, style="xxpp", verbose=False, tol=1e-10):
    """
    严格的威廉姆森分解 V = S D S^T
    返回对角辛谱 D 和辛变换 S，使得 S^T J S = J
    """
    if not isinstance(V, np.ndarray) or V.ndim != 2 or V.shape[0] != V.shape[1] or V.shape[0] % 2 != 0:
        raise ValueError("Input V must be a real, symmetric, even-dimensional matrix.")

    if not np.allclose(V, V.T, atol=tol):
        raise ValueError("V must be symmetric.")

    N = V.shape[0] // 2
    J = symplectic_typical(N, style=style)

    # 1. 构造 J V
    JV = J @ V

    # 2. 求解本征对
    eigvals, eigvecs = np.linalg.eig(JV)
    idx = np.argsort(np.abs(np.real(eigvals)))
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    # 3. 只取正的共轭成对本征值
    pos_eigs = []
    pos_vecs = []
    for i in range(len(eigvals)):
        if np.imag(eigvals[i]) > 0:
            pos_eigs.append(np.imag(eigvals[i]))
            pos_vecs.append(np.real(eigvecs[:, i]))

    nu = np.array(pos_eigs)  # 辛特征值
    D = np.diag(np.concatenate([nu, nu]))

    # 4. 构造辛正则基
    W = np.stack(pos_vecs, axis=1)
    W_tilde = J @ W
    S = np.concatenate([W, W_tilde], axis=1)

    # 5. 使用 V 恢复 S 的缩放，使得 V = S D S^T
    # 构造正交化基
    try:
        L = scipy.linalg.sqrtm(V)
        S = L @ np.linalg.inv(scipy.linalg.sqrtm(D))
    except Exception as e:
        raise RuntimeError("数值计算失败") from e
    
    if verbose:
        print("辛特征值 ν_i:", nu)
        print("D:\n", D)
        print("S^T J S - J =\n", S.T @ J @ S - J)

    return D, S

def williamson_decomposition(V, style="xxpp", verbose=False, tol=1e-10):
    """
    统一的威廉姆森分解：V = S D S^T
    支持 xxpp 或 xpxp 排列格式
    返回：
      D: 辛谱，对角矩阵（格式取决于 style）
      S: 辛矩阵
    """
    if not isinstance(V, np.ndarray) or V.ndim != 2 or V.shape[0] != V.shape[1] or V.shape[0] % 2 != 0:
        raise ValueError("V must be a symmetric positive definite matrix of even size.")
    if not np.allclose(V, V.T, atol=tol):
        raise ValueError("V must be symmetric.")

    N = V.shape[0] // 2
    J = symplectic_typical(N, style=style)

    # Step 1: i J V 的特征值
    iJV = 1j * J @ V
    eigvals = np.linalg.eigvals(iJV)
    symplectic_eigs = np.sort(np.abs(eigvals))[::2]  # 非简并

    # Step 2: 构造标准辛谱 D
    if style == "xxpp":
        D = np.diag(np.tile(symplectic_eigs, 2)) # [v1, v2, ..., v1, v2, ...]
    elif style == "xpxp":
        D = np.diag(np.repeat(symplectic_eigs, 2)) # [v1, v1, ..., v2, v2, ...]
    else:
        raise ValueError("Unknown style for Williamson decomposition.")

    # Step 3: 构造 S
    sqrtV = scipy.linalg.sqrtm(V)
    sqrtD = scipy.linalg.sqrtm(D)
    S = sqrtV @ np.linalg.inv(sqrtD)

    # # Step 4: 根据风格变换变量排列（如果 style 是 xpxp）
    # if style == "xpxp":
    #     P = permutation_matrix(N)
    #     S = P.T @ S  # 把 S 从 xxpp -> xpxp
    #     D = P.T @ D @ P

    if verbose:
        print("Style:", style)
        print("Symplectic eigenvalues ν_i:", symplectic_eigs)
        print("D =\n", D)
        print("S =\n", S)
        J_check = S.T @ J @ S
        print("S^T J S - J =\n", J_check - J)

    return D, S

def strict_williamson_decomposition(V, style="xxpp", tol=1e-10, verbose=False):
    """
    严格的威廉姆森分解：将正定协方差矩阵 V 分解为
         V = S D S^T
    其中 S 满足辛条件 S^T J S = J。
    
    算法基于下面的公式：
         S = V^(1/2) * ( V^(1/2) J V^(1/2) )^(-1/2)
         D = S^(-1) V (S^(-1))^T
         
    参数：
      V      : (2N x 2N) 实对称正定矩阵（协方差矩阵）
      style  : "xxpp" 或 "xpxp"，决定 J 的形式
      tol    : 数值判断容差
      verbose: 如果为 True，则打印中间调试信息
      
    返回：
      S      : 计算得到的辛矩阵，理论上应满足 S^T J S = J
      D      : 对角辛谱矩阵，排列形式将由 style 决定
    """
    # 检查 V 是否对称
    if not np.allclose(V, V.T, atol=tol):
        raise ValueError("V must be symmetric.")
    
    # 计算 V 的唯一正定平方根
    X = scipy.linalg.sqrtm(V)
    if np.max(np.abs(np.imag(X))) < tol:
        X = np.real(X)
    
    N = V.shape[0] // 2
    J = symplectic_typical(N, style=style)
    
    # 计算 A = V^(1/2) J V^(1/2)
    A = X @ J @ X
    
    # 求 A 的逆平方根：即计算 (A)^(-1/2)
    # 注意 A 是反对称矩阵，理论上其谱为纯虚数，但数值上可能存在小的实部误差
    A_sqrt = scipy.linalg.sqrtm(A)
    if np.max(np.abs(np.imag(A_sqrt))) < tol:
        A_sqrt = np.real(A_sqrt)
    # 取逆：注意不直接求 A^(-1/2) = inv(sqrtm(A))
    A_inv_sqrt = np.linalg.inv(A_sqrt)
    # 计算 S
    S = X @ A_inv_sqrt

    # 验证 S 是否为辛矩阵：应满足 S^T J S = J
    if verbose:
        J_check = S.T @ J @ S
        print("S^T J S - J =\n", J_check - J)
    
    # 计算 D = S^(-1) V S^(-T)
    Sinv = np.linalg.inv(S)
    D = Sinv @ V @ Sinv.T

    # 对于排列风格的控制，通常 D 的排列可通过 tile/repeat 重新排序，
    # 但由于我们是从严格分解中获得 D，
    # 此时 D 的理论形式应为 block-diag(ν1 I_2, ..., νN I_2).
    if verbose:
        print("D =\n", D)
        print("S =\n", S)
    
    return D, S

def are_symplectically_equivalent(S1, S2, style="xxpp", tol=1e-8, verbose=False):
    """
    判断两个辛矩阵是否辛等价：是否存在辛矩阵 R，使得 S2 = S1 @ R
    """
    if S1.shape != S2.shape:
        return False

    N = S1.shape[0] // 2
    J = symplectic_typical(N, style)

    # 计算 R = S1^{-1} S2
    R = np.linalg.inv(S1) @ S2

    # 判断 R 是否为辛矩阵
    check = R.T @ J @ R
    is_symplectic = np.allclose(check, J, atol=tol)

    if verbose:
        print("R = S1⁻¹ S2 =\n", R)
        print("Rᵀ J R - J =\n", check - J)

    return is_symplectic

def bloch_messiah_decomp(S, tol=1e-10):
    """
    对一个辛矩阵 S 执行 Bloch–Messiah 分解:
        S = O1 @ Sigma @ O2
    返回:
        O1, Sigma, O2
    """
    if not is_symplectic(S):
        raise ValueError("S is not symplectic")

    U, Sigma_vals, Vh = np.linalg.svd(S)

    # 构造辛兼容的对角矩阵：Sigma = diag(s_1, 1/s_1, s_2, 1/s_2, ...)
    N = S.shape[0] // 2
    Sigma_matrix = np.zeros_like(S)
    for i in range(N):
        s = Sigma_vals[2 * i]
        Sigma_matrix[2 * i, 2 * i] = s
        Sigma_matrix[2 * i + 1, 2 * i + 1] = 1 / s

    # Vh 是 V^T，我们需要 V
    V = Vh.T
    O1 = U
    O2 = V.T

    return O1, Sigma_matrix, O2

def reconstruct_from_bmd(O1, Sigma, O2):
    return O1 @ Sigma @ O2

def bloch_messiah_channel(K, N_noise, tol=1e-10):
    """
    将高斯通道表示为 Bloch-Messiah 分解 + 噪声项：
        K V K^T + N = (O1 Σ O2) V (O1 Σ O2)^T + N
    输入:
        K: 高斯通道的线性变换矩阵 (2N x 2N)
        N_noise: 噪声协方差矩阵 (2N x 2N)
    返回:
        O1, Sigma, O2, N_noise
    """
    U, s, Vh = np.linalg.svd(K)
    V = Vh.T
    Sigma = np.zeros_like(K)
    for i in range(len(s)):
        Sigma[i, i] = s[i]

    O1 = U
    O2 = V.T

    return O1, Sigma, O2, N_noise

def apply_bloch_messiah_channel(V_in, O1, Sigma, O2, N):
    """
    应用 Bloch-Messiah + 噪声形式的通道到输入协方差矩阵 V_in 上
    """
    K = O1 @ Sigma @ O2
    return K @ V_in @ K.T + N

def invert_loss_channel(V_out, eta, hbar=1.0):
    """
    已知输出协方差矩阵和损耗率，反推输入态（假设环境为真空态）
    :param V_out: 输出协方差矩阵
    :param eta: 透过率 (0 < eta <= 1)
    :param hbar: 量子单位 (默认1)
    :return: V_in (估计的输入协方差矩阵)
    """
    if not (0 < eta <= 1):
        raise ValueError("η 应该在 (0, 1] 之间")
    n = V_out.shape[0]
    I = np.eye(n)
    V_env = (hbar / 2) * I
    V_in = (V_out - (1 - eta) * V_env) / eta
    return V_in

def invert_gaussian_channel(V_out, K, N):
    """
    高斯通道反演：已知输出协方差矩阵、K 和 N，求输入态
    V_out = K V_in K^T + N
    解得：V_in = K^{-1} (V_out - N) (K^{-1})^T
    """
    K_inv = np.linalg.inv(K)
    return K_inv @ (V_out - N) @ K_inv.T

def two_mode_squeezed_V(r):
    """
    双模压缩态协方差矩阵（变量排列为 xxpp）
    ħ = 1
    :param r: 压缩参数
    :return: 2x2 协方差矩阵 V
    """
    ch = np.cosh(2 * r)
    sh = np.sinh(2 * r)
    
    V = 0.5 * np.array([
        [ ch,  sh,   0,   0],
        [ sh,  ch,   0,   0],
        [  0,   0,  ch, -sh],
        [  0,   0, -sh,  ch]
    ])
    return V

In [62]:
covar_test0_xxpp = np.array([
    [ 0.60887691, -0.19581562, 0.        , 0.08951477],
    [-0.19581562, 0.57113222,  0.10925116, 0.        ],
    [ 0.        , 0.10925116,  0.61786801, 0.21770192],
    [ 0.08951477, 0.        ,  0.21770192, 0.52902511]])
covar_test1_xxpp = np.array([
    [ 0.5, -0.2, 0., 0.],
    [-0.2, 0.5,  0., 0.],
    [ 0., 0.,  0.5, 0.2],
    [ 0., 0.,  0.2, 0.5]])
covar_test2_xxpp = np.array([
    [ 2., 0],
    [ 0, 2.]])
covar_test3_xxpp = np.array([
    [ 0.60887691, -0.19581562, 0.        , 0.08951477],
    [-0.19581562, 0.57113222,  0.10925116, 0.        ],
    [ 0.        , 0.10925116,  0.61786801, 0.21770192],
    [ 0.08951477, 0.        ,  0.21770192, 0.52902511+0.01]])

print(is_physical_covariance(covar_test3_xxpp, style="xxpp", verbose=True))

print("威廉姆森分解：")
D_sf, S_sf = sf.decompositions.williamson(covar_test0_xxpp)
print("D_sf:\n", D_sf)
print("S_sf:\n", S_sf)
print(is_symplectic(S_sf, style="xxpp", verbose=True))

print("BM分解：")
U1, S_D, U2 = sf.decompositions.bloch_messiah(S_sf)
print("U1:\n", U1)
print("S_D:\n", S_D)
print("U2:\n", U2)

print(S_D @ U2 @ D_sf @ U2.T @ S_D.T)
print(S_D @ U2 @ (0.5*np.eye(4)) @ U2.T @ S_D.T)
print(U2 @ D_sf @ U2.T)
print(U2 @ (0.5*np.eye(4)) @ U2.T)

最低特征值实部: 0.0030825801927262374
True
威廉姆森分解：
D_sf:
 [[0.57059638 0.         0.         0.        ]
 [0.         0.49794635 0.         0.        ]
 [0.         0.         0.57059638 0.        ]
 [0.         0.         0.         0.49794635]]
S_sf:
 [[ 0.91751241 -0.33325802  0.2039484   0.31527811]
 [-0.00617212  0.98899355  0.21478086 -0.34053229]
 [-0.22995354 -0.04210162  1.01374588 -0.02911616]
 [-0.11791032  0.21871263  0.38041469  0.91258873]]
S^T J S - J =
 [[ 1.95084421e-17 -3.00759823e-16  2.22044605e-16  1.17404253e-16]
 [ 3.07548544e-16  1.59618875e-18 -1.06508233e-17  0.00000000e+00]
 [-2.22044605e-16  3.49336749e-17  1.60607205e-17 -1.60837062e-16]
 [-1.14810729e-16  0.00000000e+00  1.83291409e-16  3.40109165e-17]]
True
BM分解：
U1:
 [[-0.65811629 -0.21496523 -0.19801647  0.6938749 ]
 [ 0.71859005 -0.1061197   0.10630505  0.67901854]
 [ 0.19801647 -0.6938749  -0.65811629 -0.21496523]
 [-0.10630505 -0.67901854  0.71859005 -0.1061197 ]]
S_D:
 [[1.24856553 0.         0.         0.

In [5]:
# === 示例开始 ===
r = 0.2  # 压缩参数
V_in = two_mode_squeezed_V(r)
print("输入双模压缩态协方差矩阵 V_in：\n", V_in)

# 进行威廉姆森分解
print("\n威廉姆森分解：")
D, S = strict_williamson_decomposition(V_in, verbose=True)

# 初始态不是真空
# N_thermal = np.array([
#     [0.11, 0.12, 0.13, 0.14],
#     [0.12, 0.22, 0.23, 0.24],
#     [0.13, 0.23, 0.33, 0.34],
#     [0.14, 0.24, 0.34, 0.44]
# ])
N_thermal = np.array([
    [0.11, 0.  , 0.  , 0.  ],
    [0.  , 0.22, 0.  , 0.  ],
    [0.  , 0.  , 0.33, 0.  ],
    [0.  , 0.  , 0.  , 0.44]
])
D_thermal = D + N_thermal
print("\n添加小扰动后的 D：\n", D_thermal)

# 进行压缩操作
V_thermal_in = S @ D_thermal @ S.T
print("\n压缩后的协方差矩阵 V_thermal_in\n", V_thermal_in)
print(np.linalg.eigvals(V_thermal_in))
# 进行威廉姆森分解
print("\n压缩后的威廉姆森分解：")
D_in, S_in = strict_williamson_decomposition(V_thermal_in, verbose=True)
D_sf, S_sf = sf.decompositions.williamson(V_thermal_in)
print("D_sf:\n", D_sf) 
print("S_sf:\n", S_sf)

# 经过损耗信道
eta = 0.5  # 透过率
K = np.sqrt(eta) * np.eye(4)  # 4x4 单位阵乘根η
N_vaccum = (1 - eta) * 0.5 * np.eye(4)  # 对应真空噪声输入
V_thermal_out = K @ V_thermal_in @ K.T + N_vaccum  # 经过损耗通道
print("\n经过损耗通道后的协方差矩阵 V_themal_out\n", V_thermal_out)
D_sf, S_sf = sf.decompositions.williamson(V_thermal_out)
print("D_sf:\n", D_sf)
print("S_sf:\n", S_sf)

# 进行威廉姆森分解
print("\n经过损耗通道后的威廉姆森分解：")
D_out, S_out = strict_williamson_decomposition(V_thermal_out, verbose=True)
D_sf, S_sf = sf.decompositions.williamson(V_thermal_out)
print("D_sf:\n", D_sf)
print("S_sf:\n", S_sf)

输入双模压缩态协方差矩阵 V_in：
 [[ 0.54053619  0.20537616  0.          0.        ]
 [ 0.20537616  0.54053619  0.          0.        ]
 [ 0.          0.          0.54053619 -0.20537616]
 [ 0.          0.         -0.20537616  0.54053619]]

威廉姆森分解：
S^T J S - J =
 [[ 3.07767127e-17 -3.39870965e-18 -1.11022302e-15  2.62848851e-17]
 [-8.15472391e-19  2.25639129e-17 -5.01844224e-17 -2.22044605e-16]
 [ 9.99200722e-16  5.41297046e-17  2.48298520e-17  2.44461099e-17]
 [-3.88898719e-17  2.22044605e-16 -3.11542853e-17  3.93207800e-17]]
D =
 [[ 5.00000000e-01 -7.00024136e-18 -3.28327894e-16  7.84155602e-18]
 [ 1.54168505e-17  5.00000000e-01 -9.26762457e-18 -1.96603900e-17]
 [-2.98307884e-16  2.32873066e-18  5.00000000e-01 -3.83406232e-17]
 [ 7.51394697e-18 -1.96603900e-17 -3.79872098e-17  5.00000000e-01]]
S =
 [[ 0.72129612  0.14236605 -0.72129612 -0.14236605]
 [ 0.14236605  0.72129612 -0.14236605 -0.72129612]
 [ 0.72129612 -0.14236605  0.72129612 -0.14236605]
 [-0.14236605  0.72129612 -0.14236605  0.72129612]

In [151]:
import numpy as np
from strawberryfields.decompositions import williamson

# 假设你有一个 2x2 的协方差矩阵（1个模式）
V = V_thermal_in
# 进行 Williamson 分解
D, S = williamson(V)

print("Diagonal D:\n", D)
print("Symplectic matrix S:\n", S)

# 验证分解是否成立
V_reconstructed = S @ D @ S.T
print("Reconstructed V:\n", V_reconstructed)


Diagonal D:
 [[0.61 0.   0.   0.  ]
 [0.   0.72 0.   0.  ]
 [0.   0.   0.61 0.  ]
 [0.   0.   0.   0.72]]
Symplectic matrix S:
 [[-0.031  0.008 -1.02  -0.201]
 [-0.006  0.038 -0.201 -1.019]
 [ 1.02  -0.201 -0.031 -0.008]
 [-0.201  1.019  0.006  0.038]]
Reconstructed V:
 [[ 0.664  0.273  0.     0.   ]
 [ 0.273  0.774  0.     0.   ]
 [ 0.     0.     0.664 -0.273]
 [ 0.     0.    -0.273  0.774]]


In [58]:
0.5*np.eye(4)

array([[0.5, 0. , 0. , 0. ],
       [0. , 0.5, 0. , 0. ],
       [0. , 0. , 0.5, 0. ],
       [0. , 0. , 0. , 0.5]])